In [111]:
# ##package

import requests
import traceback
import pandas as pd
from bs4 import BeautifulSoup as bs


### function

def getAuthor(table):
    authorEle   = mainTable.select("tr")[0].select("td")[0]
    author_name = authorEle.select("div.authi a")[0].text
    tdTag       = table.select("tr")[0].select("td")[0]
    
    author_post_cnt     = tdTag.select("dl.pil dd")[0].text
    author_score        = tdTag.select("dl.pil dd")[1].text.replace(" 點","")
    author_diving_value = tdTag.select("dl.pil dd")[2].text.replace(" 米","")
    
    return {
        "author_name" : author_name,
        "author_post_cnt"     : int(author_post_cnt),
        "author_score"        : int(author_score),
        "author_diving_value" : int(author_diving_value)
    }

def getContent(table):
    trTag = table.select("tr")[1]
    if trTag.i: trTag.i.extract()
    return trTag.text

def getTime(table):
    return table.select("td")[1].select("div.authi em")[0].text.replace("發表於 ","").replace(" AM","")


######################################
def getHiddenParams(url):
    
    res0  = requests.get(url)
    soup0 = bs(res0.text,"lxml")

    cookietime = soup0.select("input[name='cookietime']")[0]["value"]
    formhash   = soup0.select("input[name='formhash']")[0]["value"]
    # print(formhash)
    
    
    return {"cookietime" : cookietime, "formhash" : formhash}

######################################
def login(formhash, cookietime):
    
    loginUrl = "https://www.eyny.com/member.php?mod=logging&action=login&loginsubmit=yes&loginhash=LIhj1&inajax=1"

    payload = {
        "formhash"            : formhash,
        "referer"             : "https://www.eyny.com/thread-12290829-1-GU7Y04C7.html",
        "loginfield"          : "username",
        "username"            : "7003un",
        "password"            : "0939771419",
        "questionid"          : "0",
        "answer"              : "",
        "cookietime"          : cookietime,
        "g-recaptcha-response": "03AOLTBLSQ9n63zcqqPQQA5FCEohXZtKD76G2DoV6_HNsggOVi4BBFl7lxBDZLlCL3kyXWYbAIiRBuXN6L9e_qk1c5-R1B4noiWkEwx_IrKRP8Oshlp5N-P_J0EoikEfTMlrRjjLy59ox7PRyxoYWR9UN5cRG_5BT7iSGeeSJagwB1YZ710EbM9rqJKDrMnDzMtv9q6aL7omX0sWCu07SNZO7r81kInV8DhD7F3AMdchIZ8Y6TH-hvRUIOHnVNWG4yDo0m1TvtyUwHnDuVjq-d6sGJfSUGouoOabss2InPrhp321m79N6RUGM"
    }

    loginHeaders = {
        "Host": "www.eyny.com",
        "Origin": "https://www.eyny.com",
        "Referer": "https://www.eyny.com/member.php?mod=logging&action=login",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }

    sess = requests.session()
    res = sess.post(loginUrl , headers = loginHeaders , data = payload)
    # print(res.text)
    # print(res.cookies.get_dict())
    
    return sess
######################################


def crawlArticleList(sess):

    # Step 2
    ### 取得文章列表
    baseUrl = "https://www.eyny.com/"
    forum = "forum-1710-1.html"
    links = []

    headers = {
        "Host": "www.eyny.com",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
    }
    res  = sess.get(baseUrl+forum , headers = headers)
    soup = bs(res.text , "lxml") 

    subLinks = [ baseUrl+ele["href"] for ele in soup.select("div#threadlist table[summary] th.new a.xst")]
    links += subLinks


    # links
    nextLinks = [baseUrl+a["href"] for a in soup.select("div.pg a")[1:10]]
    lastLink  = baseUrl+forum

    for nextLink in nextLinks:
        headers["Refer"] = lastLink

        res  = s.get(nextLink , headers = headers)
        subSoup = bs(res.text , "lxml") 

        subLinks = [ baseUrl+ele["href"] for ele in subSoup.select("div#threadlist table[summary] th.common a.xst")]
        links += subLinks

        lastLink = nextLink
        
    return links

######################################
def crawlArticleDetail(sess,links):

    dataList = []
    for l in links:
        try:
    #     res2  = requests.get(l,headers=headers)
            res2  = sess.get(l,headers=headers)
            soup2 = bs(res2.text,"lxml")
            tableHeader = soup2.select("div#postlist > table")[0]

            pageview = tableHeader.select("td")[0].select("span")[1].text
            resp_cnt = tableHeader.select("td")[0].select("span")[4].text
            title    = tableHeader.select("td")[1].text.replace("[複製鏈接]","")

            tables = soup2.select("div#postlist > div > table")
            mainTable = tables[0]

            author   = getAuthor(mainTable)
            content  = getContent(mainTable)
            time     = getTime(mainTable)

            data = {
                "title" : title,
                "time"  : time,
                "content" : content,
                "pageview": pageview,
                "resp_cnt": resp_cnt,
                "author_name"          : author["author_name"],
                "author_post_cnt"      : author["author_post_cnt"],
                "author_score"         : author["author_score"],
                "author_diving_value"  : author["author_diving_value"],
                "resp": []
            }

            for table in tables[1:]:
                respAuthor = getAuthor(table)
                resp_data = {
                    "time"                : getTime(table),
                    "content"             : getContent(table),
                    "author_name"         : respAuthor["author_name"],
                    "author_post_cnt"     : respAuthor["author_post_cnt"],
                    "author_score"        : respAuthor["author_score"],
                    "author_diving_value" : respAuthor["author_diving_value"],
                }

                data["resp"].append(resp_data)


            dataList.append(data)

            ### 檢查是否文章有效
            print(data["title"])
            print("="*80)
        except:
            print("*"*80)
            traceback.print_exc()
            print("*"*80)
            
    return dataList
######################################

def saveFile(dataList,name):

    df = pd.DataFrame(dataList)
    df = df[["title","time","resp_cnt","pageview","content","author_name","author_post_cnt","author_score","author_diving_value","resp"]]
    df.to_excel(name,index=False)

    
    
### main    
# Step 0
### 取得隱藏變數
loginPageUrl = "https://www.eyny.com/member.php?mod=logging&action=login"   
hiddenParams = getHiddenParams(loginPageUrl)
print("Step 0 : 取得隱藏變數 ok !!!")


## Step 1
### 登入

sess = login(hiddenParams["formhash"], hiddenParams["cookietime"])
print("Step 1 : 登入 ok !!!")



# Step 2
    ### 取得文章列表
links = crawlArticleList(sess)    
print("Step 2 : 文章列表 ok !!!")



# Step3
### 抓取文章內容
# l = links[8]

datalist = crawlArticleDetail(sess,links[:10])

print("Step 3 : 抓取文章內容 ok !!!")


# Step4
### 存資料

name= "20190810-sample.xlsx"
saveFile(dataList, name)
print("Step 4 : 存成 excel ok !!!")


Step 0 : 取得隱藏變數 ok !!!
Step 1 : 登入 ok !!!
Step 2 : 文章列表 ok !!!
[綜合討論]死靈法師要出了嗎?
********************************************************************************
********************************************************************************


Traceback (most recent call last):
  File "<ipython-input-111-14886afc2889>", line 150, in crawlArticleDetail
    respAuthor = getAuthor(table)
  File "<ipython-input-111-14886afc2889>", line 11, in getAuthor
    author_post_cnt     = tdTag.select("dl.pil dd")[0].text
IndexError: list index out of range


********************************************************************************
********************************************************************************


Traceback (most recent call last):
  File "<ipython-input-111-14886afc2889>", line 150, in crawlArticleDetail
    respAuthor = getAuthor(table)
  File "<ipython-input-111-14886afc2889>", line 11, in getAuthor
    author_post_cnt     = tdTag.select("dl.pil dd")[0].text
IndexError: list index out of range


[綜合討論]我的第一貼~算人品啦
[玩家互動]一顆頭的寵物:惡靈火顱
[玩家互動]炫耀文囉 因為太開心了 
[綜合討論]賽季18會不少傳奇會增加特效, 大家要回鍋嗎?
[綜合討論]暗黑破壞神快要被放棄了嗎?
[綜合討論]死靈法師技能試玩效果動畫分享 
[綜合討論]聖教軍推薦玩法?
Step 3 : 抓取文章內容 ok !!!
Step 4 : 存成 excel ok !!!


In [14]:
print(df.dtypes)
df = df.astype({
    "pageview" : "int32",
    "author_name" : "str",
    "resp_cnt" : "int32"
})
df.dtypes

title                  object
time                   object
resp_cnt                int32
pageview                int32
content                object
author_name            object
author_post_cnt         int64
author_score            int64
author_diving_value     int64
resp                   object
dtype: object


title                  object
time                   object
resp_cnt                int32
pageview                int32
content                object
author_name            object
author_post_cnt         int64
author_score            int64
author_diving_value     int64
resp                   object
dtype: object

In [57]:
data1 = df[df["title"].str.contains("死靈")]
# print(data1)
data1.count()

# 2.熱門文章
# df[df["pageview"]>50000]

# 3.活躍用戶 意見領袖
data2 = df[ (df["author_post_cnt"]>1016) & (df["author_score"]>500)]
data2["author_name"]

# print(list(data2["author_name"]))

# authors = {}

# for a in list(data2["author_name"]):
#     if a in authors:
#         authors[a] +=1
#     else:
#         authors[a] = 1 
        
# # print(authors)

# authors

# data2[data2["author_name"] == "NANABOBO2003"]
# groupby
data2.groupby("author_name").size()
data2.groupby("author_name").get_group("NANABOBO2003")


# 4. 各種命題
# 1) 文章分幾類,每類個數各多少
df.groupby("title").size()
df["abc"] = 123456
df["qqq"] = df["author_post_cnt"]*100
df["catagory"] = df["title"].str[1:5]

data3 = df.groupby("catagory")

# 2) 啊討論什麼
data3.get_group("綜合討論")

# 3)近期主題是什麼?b
data4 = data3.get_group("綜合討論")

data4[ data4["time"] > "2019-5-8" ][3:4]["resp"].values[0]

[{'time': '2019-7-15 04:12 PM',
  'content': '看人摟\r\n那就好好玩2等四吧',
  'author_name': 'sangelva',
  'author_post_cnt': 566,
  'author_score': 1,
  'author_diving_value': 12381},
 {'time': '2019-7-16 12:48',
  'content': '只打賽季拿箱子阿~\r\n好像變不太出新把戲囉',
  'author_name': 'sangelva',
  'author_post_cnt': 566,
  'author_score': 1,
  'author_diving_value': 12381},
 {'time': '2019-7-16 07:04 PM',
  'content': '2真的才是神作',
  'author_name': 'sangelva',
  'author_post_cnt': 566,
  'author_score': 1,
  'author_diving_value': 12381},
 {'time': '2019-7-17 02:12 PM',
  'content': '唉 已經不玩很久囉 還是慢慢等4代出來再看看了 想當初剛上市時 也是各大便利商店跑 就為了買一包實體包說 ',
  'author_name': 'sangelva',
  'author_post_cnt': 566,
  'author_score': 1,
  'author_diving_value': 12381},
 {'time': '2019-7-17 02:18 PM',
  'content': '只有新賽季會玩上半個月吧，也沒新傳奇只加個特效就要撐幾個月，內容太貧乏',
  'author_name': 'sangelva',
  'author_post_cnt': 566,
  'author_score': 1,
  'author_diving_value': 12381},
 {'time': '2019-7-17 11:42 PM',
  'content': '老梗遊戲\xa0 \xa0有空就拿出來玩玩\xa0 \xa0當單機

In [97]:
# content 含有D3 or 暗黑破壞神3 的 pageview合計
data7 = df[(df["title"].str.contains("D3")) | (df["title"].str.contains("暗黑破壞神3"))]
print(sum(list(data7["pageview"])))
for i in list(data7["pageview"]):
    print(type(i))
allpageview = 0
for pageview in list(data7["pageview"]):
    allpageview+= pageview
    
print(allpageview)

123407
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
123407


In [110]:
# 時間在2018-10-01 - 2019-05-01 的作者 的文章 個數統計
data5 = df[ (df["time"] > "2018-10-1") & (df["time"] < "2019-5-1")]
data5["author_name"]
data6 = data5.groupby("author_name").size().sort_values(ascending = False)
# print(data6)
print(data6.keys())
print(list(data6))

Index(['kyo111111', 'herbert111', 'severus', 'a751106', 'andy0048', 'kkcc1103',
       'daishan39', 'milhouse1112', 'mr0kimo', 'bbc111', 'notoriousbs', '遊人舊夢',
       'sangelva', '寂靜之聲', 'abyss2', 'edward0606a', 'forceedge55',
       'flytomit2017', 'eric6311', 'envy99', 'egg-11111', 'TENNOH', 'atpx2',
       'dttvawb', 'a239211', 'a552565100', 'aa44747', 'happy760519',
       'aa953690', 'basara30', 'jkr1428', 'hhhhh193', 'ilkid02', 'yj-climax',
       'xxguavaxx', 'wl00089554', 'voter00000000', 's0926876620', 'raix',
       'qwe_789963', 'papaay', 'oligokof2', 'nathan6001', 'mtf931', 'longear6',
       'liuey', 'kkkiori', 'kaien456', '蕭神經', 'iphoneGG', '0937687494'],
      dtype='object', name='author_name')
[16, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [34]:
data2.groupby("author_name").size()

author_name
HLPT             5
Lhellraiser      1
NANABOBO2003    15
QWOP19840        1
atpx2            2
boy61012         1
callhata         1
cool6084         1
fightxp          1
jason3340        1
jkr1428          1
kkkiori          2
liuey            2
maligab          1
mashall1         3
nathan6001       1
s0602002         1
yen790914        1
zakayara         2
dtype: int64

In [20]:
sorted(list(df["author_post_cnt"]))

[0,
 4,
 5,
 7,
 8,
 10,
 10,
 11,
 12,
 12,
 13,
 14,
 15,
 15,
 15,
 15,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 19,
 20,
 22,
 22,
 22,
 22,
 22,
 22,
 25,
 27,
 30,
 30,
 30,
 32,
 39,
 49,
 58,
 65,
 78,
 78,
 81,
 81,
 82,
 82,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 94,
 99,
 101,
 105,
 108,
 109,
 111,
 120,
 120,
 125,
 129,
 134,
 136,
 142,
 146,
 153,
 153,
 174,
 174,
 181,
 183,
 183,
 189,
 193,
 194,
 194,
 201,
 204,
 204,
 204,
 204,
 210,
 210,
 210,
 210,
 210,
 210,
 210,
 221,
 231,
 241,
 241,
 241,
 241,
 241,
 245,
 258,
 258,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 262,
 264,
 266,
 273,
 295,
 313,
 317,
 334,
 341,
 352,
 352,
 369,
 391,
 391,
 391,
 391,
 405,
 406,
 429,
 433,
 446,
 446,
 446,
 446,
 446,
 459,
 480,
 506,
 530,
 566,
 566,
 566,
 566,
 618,
 622,
 636,
 641,
 641,
 641,
 669,
 669,
 669,
 669,
 669,
 669,
 669,
 697,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,
 726,